   # Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_test_data[-5:]

array([[5793, 2345,    0,    2],
       [5793, 2358,    0,    4],
       [5793, 2369,    0,    1],
       [5793, 2399,    0,    3],
       [5793, 2483,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258332
Nr of rows in test file 117774


In [8]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Nr of emails in training file 4013


In [9]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1724


### How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [14]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, 
                     word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array,
    nr_words -- size of vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column.
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column.
    cat_idx -- position of the label (spam is 1, nonspam is 0) id in the sparse matrix. Default: 3rd column.
    freq_idx -- position of occurence of word in the sparse matrix. Default: 4th column.
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
        
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix
    

In [15]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 8.53 s


In [16]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


# Train the Naive Bayes Model

## Calculating the Probability of Spam

In [18]:
# Probability of Spam Emails in the Training DataSet

prob_spam = round(full_train_data['CATEGORY'].value_counts()[1] / full_train_data['CATEGORY'].value_counts().sum(), 2)
prob_spam

0.31

## Total Number of Words / Tokens

In [19]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [21]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [22]:
total_wc = email_lengths.sum()
total_wc

429184

## Number of Tokens in Spam & Ham Emails

In [23]:
# spam emails
spam_emails = full_train_data[full_train_data.CATEGORY == 1]
# spam emails without CATEGORY column
full_spam_features = spam_emails.loc[:, full_train_data.columns != 'CATEGORY']
# number of words per email
spam_lengths = full_spam_features.sum(axis=1)
# total number of words in spam emails
spam_wc = spam_lengths.sum()
spam_wc

176318

In [24]:
# non-spam emails
ham_emails = full_train_data[full_train_data.CATEGORY == 0]
# non-spam emails without CATEGORY column
full_ham_features = ham_emails.loc[:, full_train_data.columns != 'CATEGORY']
# number of words per email
ham_lengths = full_ham_features.sum(axis=1)
# total number of words in non-spam emails
ham_wc = ham_lengths.sum()
ham_wc

252866

In [25]:
round(spam_lengths.mean())

141

In [26]:
round(ham_lengths.mean())

91

In [27]:
spam_lengths.describe()

count    1248.000000
mean      141.280449
std       264.821351
min         1.000000
25%        43.000000
50%        76.000000
75%       150.000000
max      4881.000000
dtype: float64

In [28]:
ham_lengths.describe()

count    2765.000000
mean       91.452441
std       246.468386
min         1.000000
25%        29.000000
50%        54.000000
75%        91.000000
max      6869.000000
dtype: float64

In [29]:
print('Average nr of words in spam emails', round(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails', round(ham_wc / ham_lengths.shape[0], 3))

Average nr of words in spam emails 141
Average nr of words in ham emails 91.452


## Summing the Tokens Occuring in Spam

In [30]:
full_train_features.shape

(4013, 2500)

In [31]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_spam_tokens.shape

(1248, 2500)

In [34]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 
summed_spam_tokens.shape

(2500,)

In [35]:
summed_spam_tokens.tail()

2495     8
2496     1
2497     2
2498     1
2499    26
dtype: int64

## Summing the Tokens Occuring in Ham

In [36]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]

summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 
summed_ham_tokens.shape

(2500,)

In [37]:
summed_ham_tokens.tail()

2495    27
2496     6
2497    28
2498    25
2499     3
dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Emails is Spam

In [38]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012186
1    0.005229
2    0.006806
3    0.011308
4    0.006817
dtype: float64

In [39]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Emails is NonSpam

In [40]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_ham[:5]

0    0.021475
1    0.010142
2    0.008008
3    0.003673
4    0.006313
dtype: float64

In [41]:
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that Token Occurs

In [42]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [43]:
prob_tokens_all.sum()

1.0

In [44]:
prob_tokens_all[-5:]

2495    0.000077
2496    0.000012
2497    0.000065
2498    0.000056
2499    0.000063
dtype: float64

# Save the Trained Model 

In [45]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [46]:
%%time
# Creating a full matrix from the sparse_test_data
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 3.74 s


In [47]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [48]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [49]:
y_test[:5]

DOC_ID
8     1
12    1
14    1
15    1
17    1
Name: CATEGORY, dtype: int64

In [50]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [51]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)